Any investor does not want to take risk and want to make max profit possible. Speaking of which, we have to say there is a tradeoff between those quantities. Research says one should not see risk and return as separate things. More risk you carry, more profit you should expect and much more you can make will be answers by Markowitz portfolio optimization (MPO).

- Given the portfolio (MPO) assign weights of each security
- it weights in a way that we get max profit with given amount of risk (everyone has different appetite for risk).

We shall try to assign weights to securities (Tesla, Google, Meta, Microsoft) to optimize the return based on the given risk. All this will be done using Markowitz Portfolio Optimization, aka Modern Port. Theory, invented by Harry Markowitz. We shall do all this using two approaches,

1. Monte Carlo simulation
2. Finding out minimal -- Using SciPy 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import plotly_express as px
import pandas_datareader as web
from datetime import datetime as dt, timedelta as td

# plt.rc\\\

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install yfinance


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.7/104.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.2/948.2 kB 25.5 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.0/508.0 kB 27.4 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.8-py3-none-any.whl size=138965 sha256=4fcabc08eb015e70dd3bce6c24f935b9617e13d11ae736564c9ebc337af9b238
  Stored in directory: /root/.cache/pip/wheels/2d/8f/b5/28f0f41351420da5423424fc74e55219347e0bb0cbca09bb82
Successfully built peewee
  Attempting uninstall: pytz
    Found existing installation: pytz 2022.1
    Uninstalling pytz-2022.1:
      Successfully uninstalled pytz-2022.1
  Attempting uninstall: requests
    Found existing installation: requests 2.28.1
    

In [3]:
import yfinance as yf
import pandas as pd
from datetime import datetime as dt, timedelta as td

# Set the start and end dates
end = dt.today()
start = end - td(days=2000)

# List of stock symbols
stocks = ['AMZN', 'TSLA', 'META', 'MSFT']

# Fetch data using yfinance
df = yf.download(stocks, start=start, end=end)

# Display the first few rows of the dataframe
print(df.head(100))


[*********************100%***********************]  4 of 4 completed

Price            Close                                           High  \
Ticker            AMZN        META        MSFT       TSLA        AMZN   
Date                                                                    
2019-07-19   98.225998  197.605164  130.059448  17.212000   99.800003   
2019-07-22   99.281502  201.550079  131.782547  17.045334   99.449997   
2019-07-23   99.724503  201.589951  132.601257  17.344667   99.889503   
2019-07-24  100.040497  203.881195  133.962585  17.658667  100.065002   
2019-07-25   98.691002  199.946228  133.458038  15.254667  100.059998   
...                ...         ...         ...        ...         ...   
2019-12-03   88.498001  198.063416  143.098694  22.413334   88.643501   
2019-12-04   88.034500  197.953827  143.616211  22.202000   89.454498   
2019-12-05   87.024002  198.601364  143.692886  22.024668   88.175003   
2019-12-06   87.580002  200.284912  145.437241  22.392668   87.720001   
2019-12-09   87.475502  200.573807  145.063416  22.

In [4]:
end = dt.today()
start = end - td(days=2000)
stocks = 'AMZN TSLA META MSFT'.split()
df = web.get_data_yahoo(stocks, start, end)

AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
# remove multilevel
df.columns = ['_'.join(col) for col in df.columns]

# Just study adj close 
df = df.iloc[:,:4]

In MPT, Close prices are the primary input because they represent the final consensus for a trading day and are essential for calculating returns.

In [ ]:
df

In [ ]:
pip install plotly

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt

# Normalize the data by dividing each stock price by its value on the first day
normalized_df = df * 100 / df.iloc[0]

# Plot the data
plt.figure(figsize=(10,6))
for column in normalized_df.columns:
    plt.plot(normalized_df.index, normalized_df[column], label=column)

plt.title("Normalized Stock Prices (Starting from 100)")
plt.xlabel("Date")
plt.ylabel("Normalized Price (%)")
plt.legend()
plt.grid(True)
plt.show()


TESLA DOING VOLATILE

In [ ]:
#px.line(df * 100 / df.iloc[0])

In [ ]:
ret_port = df.pct_change()
px.line(ret_port)

Looks like TSLA is having a jumpy ride, as you might have thought :)

In [ ]:
np.random.seed(1)
# Weight each security
weights = np.random.random((4,1))
# normalize it, so that some is one
weights /= np.sum(weights)
print(f'Normalized Weights : {weights.flatten()}')

# We generally do log return instead of return
log_ret = np.log(df / df.shift(1))
log_ret

# Expected return (weighted sum of mean returns). Mult by 252 as we always do annual calculation and year has 252 business days
exp_ret = log_ret.mean().dot(weights)*252 
print(f'\nExpected return of the portfolio is % : {(exp_ret[0])*100}')

# Exp Volatility (Risk)
exp_vol = np.sqrt(weights.T.dot(252*log_ret.cov().dot(weights)))
print(f'\nVolatility of the portfolio %: {(exp_vol[0][0])*100}')

# Sharpe ratio
sr = exp_ret / exp_vol
print(f'\nSharpe ratio of the portfolio: {sr[0][0]}')

# Monte Carlo Simulation

We basically randomly initialize weights of the securities and check the returns vs risks. And among these set of weights we will find the one, for given risk, which gives the maximum return. However, one can do all this solving optimization problem, using SciPy. 

In [ ]:
# number of simulation
n = 50_000
# n = 10

port_weights = np.zeros(shape=(n,len(df.columns)))
port_volatility = np.zeros(n)
port_sr = np.zeros(n)
port_return = np.zeros(n)

num_securities = len(df.columns)
# num_securities
for i in range(n):
    # Weight each security
    weights = np.random.random(4)
    # normalize it, so that some is one
    weights /= np.sum(weights)
    port_weights[i,:] = weights 
    #     print(f'Normalized Weights : {weights.flatten()}')

    # Expected return (weighted sum of mean returns). Mult by 252 as we always do annual calculation and year has 252 business days
    exp_ret = log_ret.mean().dot(weights)*252 
    port_return[i] = exp_ret
#     print(f'\nExpected return is : {exp_ret[0]}')

    # Exp Volatility (Risk)
    exp_vol = np.sqrt(weights.T.dot(252*log_ret.cov().dot(weights)))
    port_volatility[i] = exp_vol
#     print(f'\nVolatility : {exp_vol[0][0]}')

    # Sharpe ratio
    sr = exp_ret / exp_vol
    port_sr[i] = sr
#     print(f'\nSharpe ratio : {sr[0][0]}')

In [ ]:
# Index of max Sharpe Ratio
max_sr = port_sr.max()
ind = port_sr.argmax()
# Return and Volatility at Max SR
max_sr_ret = port_return[ind]
max_sr_vol = port_volatility[ind]


In [ ]:
plt.figure(figsize=(20,15))
plt.scatter(port_volatility,port_return,c=port_sr, cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility', fontsize=15)
plt.ylabel('Return', fontsize=15)
plt.title('Efficient Frontier (Bullet Plot)', fontsize=15)
plt.scatter(max_sr_vol, max_sr_ret, c='blue', s=150, edgecolors='red', marker='o', label='Max \
Sharpe ratio Portfolio')
plt.legend();

In [ ]:
for weight, stock in zip(port_weights[ind],stocks):
    print(f'{round(weight * 100, 2)} % of {stock} should be bought.')
    
# best portfolio return
print(f'\nMarkowitz optimal portfolio return is : {round(max_sr_ret * 100, 2)}% with volatility \
{max_sr_vol}')

All we did above can be done using optimization, which we do next.

# SciPy to get the max of Sharpe Ration

In [ ]:
from scipy import optimize

In [ ]:
log_mean = log_ret.mean() * 252
cov = log_ret.cov() * 252

In [ ]:
# Some helper functions
def get_ret_vol_sr(weights):
    weights = np.array(weights)
    ret = log_mean.dot(weights)
    vol = np.sqrt(weights.T.dot(cov.dot(weights)))
    sr = ret / vol
    return np.array([ret, vol, sr])

# Negate Sharpe ratio as we need to max it but Scipy minimize the given function
def neg_sr(weights):
    return get_ret_vol_sr(weights)[-1] * -1

# check sum of weights 
def check_sum(weights):
    return np.sum(weights) - 1

# Constraints for the optimization problem
cons = {'type':'eq','fun':check_sum}
# bounds on weights
bounds = ((0,1),(0,1),(0,1),(0,1))
# initial guess for optimization to start with
init_guess = [.25 for _ in range(4)]


# Call minimizer
opt_results = optimize.minimize(neg_sr, init_guess, constraints=cons, bounds=bounds, method='SLSQP')


In [ ]:
optimal_weights = opt_results.x
# optimal_weights
for st, i in zip(stocks,optimal_weights):
    print(f'Stock {st} has weight {np.round(i*100,2)} %')

In [ ]:
mc_weights = port_weights[ind]
for st, i in zip(stocks,mc_weights):
    print(f'Stock {st} has weight {np.round(i*100,2)} %')

In [ ]:
# Comparing two results we see that we get very close results
(optimal_weights - mc_weights)

In [ ]:
get_ret_vol_sr(optimal_weights), get_ret_vol_sr(mc_weights)

print('For a given portfolio we have: (Using SciPy optimizer)\n \n')
for i, j in enumerate('Return Volatility SharpeRatio'.split()):
    print(f'{j} is : {get_ret_vol_sr(optimal_weights)[i]}\n')
    
print('For a given portfolio we have: (Using Monte Carlo)\n \n')
for i, j in enumerate('Return Volatility SharpeRatio'.split()):
    print(f'{j} is : {get_ret_vol_sr(mc_weights)[i]}\n')

So MC and Optimizer gives very close results. MC is good, but if we have many assets, since MC computationally heavy, SciPy is going to be a saviour!!!

# Frontier curve

Best return for given volatility or vice versa.

In [ ]:
frontier_y = np.linspace(port_return.min(), port_return.max(), 100)
frontier_vol = []

def minimize_vol(weights):
    return get_ret_vol_sr(weights)[1]

for possible_ret in frontier_y:
    cons = ({'type':'eq','fun':check_sum},
            {'type':'eq','fun':lambda w:get_ret_vol_sr(w)[0] - possible_ret})
    result = optimize.minimize(minimize_vol, init_guess, method='SLSQP', constraints=cons, bounds=bounds)
    frontier_vol.append(result['fun'])    

In [ ]:
plt.figure(figsize=(20,15))
plt.scatter(port_volatility,port_return,c=port_sr, cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility', fontsize=15)
plt.ylabel('Return', fontsize=15)
plt.title('Efficient Frontier', fontsize=15)
plt.scatter(max_sr_vol, max_sr_ret, c='blue', s=150, edgecolors='red', marker='o')

plt.plot(frontier_vol, frontier_y, c='magenta', ls='--', lw=3, label='Efficient Frontier')
plt.legend();

It is common practice to choose the portfolio with max sharpe ratio. Thought one can always make a choice as per one's taste. Efficient frontier gives us max profit for given amount of risk. So anything below the given efficient frontier will be a bad choice (of course, we are assuming above the risk free rate).

CAPM, Beta and Sharpe ratio are tools to evaluate the stocks too. One may see,
https://www.kaggle.com/code/bhavinmoriya/beta-of-a-stock-capm-from-scratch